In [7]:
import pandas as pd

In [8]:
data = pd.DataFrame(pd.read_excel("/Users/paul/Desktop/Eulith_AI_Agent/annotation_sets/WETH_UNI_SWAP.xlsx"))

In [9]:
#0 to 99 : Swap WETH to UNI
#100 to 208 : Swap UNI to WETH
#209 tà 317 : Swap USDC to WETH
data

,"""Set up a WETH and UNI swap"""
0,"""Initiate a swap between WETH and UNI"""
1,"""Perform a WETH to UNI swap"""
2,"""Exchange WETH for UNI"""
3,"""Swap WETH with UNI"""
4,"""Conduct a WETH-UNI swap"""
...,...
204,"""Set in motion a simple UNI-WETH trade"""
205,"""Kick off an easy UNI and WETH swap"""
206,"""Launch a no-nonsense UNI-WETH exchange"""
207,"""Begin a straightforward UNI and WETH trade"""
